Este archivo principal une todos los módulos y ejecuta el entrenamiento, validación y evaluación del modelo GCN.

In [34]:
#Importamos los modulos necesarios

import torch
from torch_geometric.loader import DataLoader
from Dataset_loader import PCBDataset  # Asegúrate que Dataset_loader se encuentra en el mismo directorio
from gcn_model import GCN
from data_loader import load_pcb_dataset
from train_eval import train, evaluate
import os

In [35]:
# Configuración del dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Mostrar mensaje de inicio
print("Iniciando el script...")
print(f"Usando dispositivo: {device}")

Iniciando el script...
Usando dispositivo: cuda


In [36]:
# Ruta al archivo de dataset
file_path = 'PCB_dataset.pth'  # Asegúrate que este archivo está en la carpeta actual

# Cargar el dataset
graphs = load_pcb_dataset(file_path)
print(f"Dataset cargado correctamente. Número de gráficos: {len(graphs)}")


Dataset cargado correctamente. Número de gráficos: 20000


In [37]:
# División del dataset
train_size = int(0.8 * len(graphs))
val_size = int(0.1 * len(graphs))
test_size = len(graphs) - train_size - val_size

train_dataset = graphs[:train_size]
val_dataset = graphs[train_size:train_size + val_size]
test_dataset = graphs[train_size + val_size:]

# Crear DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [38]:
# Definición del modelo
input_dim = 1  # Potencia como única característica de entrada
hidden_dim = 64
output_dim = 1  # Temperatura predicha por cada nodo
num_layers = 3
learning_rate = 0.001
epochs = 10

# Crear el modelo
model = GCN(input_dim, hidden_dim, output_dim, num_layers).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print("Modelo creado correctamente.")


Modelo creado correctamente.


In [39]:
# Entrenamiento del modelo
print("Iniciando entrenamiento...")
for epoch in range(epochs):
    train_loss = train(model, train_loader, optimizer, device)
    val_mse, val_mae, val_r2, val_accuracy = evaluate(model, val_loader, device)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f} - Val MSE: {val_mse:.4f} - Val MAE: {val_mae:.4f} - Val R2: {val_r2:.4f} - Val Accuracy: {val_accuracy:.2f}%")



Iniciando entrenamiento...


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5408x2 and 1x64)

In [ ]:
# Evaluación en el conjunto de prueba
print("Evaluando en el conjunto de prueba...")
error_threshold = 5.0  # Cambia esto al valor que desees en Kelvin

test_mse, test_mae, test_r2, test_accuracy = evaluate(model, test_loader, device, error_threshold=5.0)


print(f"Test MSE: {test_mse:.4f} - Test MAE: {test_mae:.4f} - Test R2: {test_r2:.4f}")
print(f"Porcentaje de nodos correctamente predichos dentro de un error de ±{error_threshold} K: {test_accuracy:.2f}%")



Evaluando en el conjunto de prueba...
Test MSE: 635.4674 - Test MAE: 20.1651 - Test R2: 0.5010
Porcentaje de nodos correctamente predichos dentro de un error de ±5.0 K: 15.74%
